In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [7]:
N = 600
dx = 12.5e-6
L = N * dx

lam = 532e-9
k = 2 * np.pi / lam
z = -100e-3

x = np.linspace(-L/2, L/2, N)
y = x
X, Y = np.meshgrid(x, y)

In [8]:
def fresnel(image, lam, z):
    coeff = np.exp(1j * k * z) / (1j * lam * z)
    kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
    transfer = np.fft.fftshift(np.fft.fft2(kernel))

    f_image = np.fft.fftshift(np.fft.fft2(image))
    f_image = f_image * transfer
    image = np.fft.ifft2(np.fft.ifftshift(f_image))
    image = np.fft.fftshift(image) # Don't know why do this but this should be exist
    return image

In [27]:
image = Image.open('images/cat_illust.jpg').convert('L')
image = np.array(image)
image = cv2.resize(image, dsize=(N,N))
# image = 255 - image

coeff = np.exp(1j * k * z) / (1j * lam * z)
kernel = coeff * np.exp(1j * k / 2 / z * (X**2 + Y**2))
transfer = np.fft.fftshift(np.fft.fft2(kernel))

f_image = np.fft.fftshift(np.fft.fft2(image))
f_image = f_image * transfer
image = np.fft.ifft2(np.fft.ifftshift(f_image))
image = np.fft.fftshift(image) # Don't know why do this but this should be exist

phase = np.angle(image)
phase = (phase - np.min(phase)) / (np.max(phase) - np.min(phase))
phase = phase * 255
Image.fromarray(phase.astype('uint8')).save('test.png')

In [29]:
from lucam import Lucam

# ‼️ 실제 카메라 촬영 로직
camera = Lucam()
capture = camera.TakeSnapshot()
capture = cv2.resize(capture, dsize=(N, N))
cv2.imwrite('fresnel_image_5.png', capture)

KeyboardInterrupt: 